In [5]:
'''
import pandas as pd
import sys
from ast import literal_eval
from sklearn.model_selection import train_test_split
Linear model implementation in tensorflow
'''
import pandas as pd
import tensorflow as tf
from chord2vec.linear_models import data_processing as dp
import numpy as np
import random
import importlib
importlib.reload(dp)
import time
from ast import literal_eval
from sklearn.model_selection import train_test_split
import gc
import pickle

會把資料整理成train_chords、test_chords、valid_chords三個lists <br>
整理過的話直接跳到 
# 儲存train_chords、test_chords、valid_chords
此chunk，以製作train_set來練chord_embedding

In [3]:
song_train_merged=pd.read_csv('../../data/song_train_merged_transferkey0425.csv',encoding="ISO-8859-1")

In [4]:
df=song_train_merged.copy()
df['song_chord']=df['song_chord'].apply(literal_eval)
df['chord']=df['chord'].apply(literal_eval)
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_name,song_detector_chord,song_chord
0,0,0,28,5cada0049fc5af34a460ae05,10cc,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,...",0,9,https://tabs.ultimate-guitar.com/tab/10cc/art_...,Art For Arts Sake,Ver 1,Art For Arts Sake 10cc,"[{'st': 0, 'et': 0.511, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,..."
1,1,1,29,5cada0049fc5af34a460ae07,10cc,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m...",0,3,https://tabs.ultimate-guitar.com/tab/10cc/baro...,Baron Samedi,Ver 1,Baron Samedi 10cc,"[{'st': 0, 'et': 3.413, 'ochord': 'A:min'}, {'...","[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:..."


In [5]:
#資料集中的所有和絃類別
chord_list=[]
for row in df.chord:
    for chord in row:
        chord_list.append(chord)
for row in df.song_chord:
    for chord in row:
        chord_list.append(chord)
chord_list=set(chord_list)
chord_list


{'A:maj',
 'A:min',
 'Ab:maj',
 'Ab:min',
 'B:maj',
 'Bb:maj',
 'Bb:min',
 'C:maj',
 'D:maj',
 'Db:maj',
 'Db:min',
 'E:maj',
 'E:min',
 'Eb:maj',
 'Eb:min',
 'F:maj',
 'F:min',
 'G:maj',
 'Gb:maj',
 'Gb:min',
 'N'}

In [6]:
del chord_list
gc.collect()

401

# Chord2Vec:
1.準備好train_data，每一個元素是一個和弦，記錄其onset的index <br>
例:以12-vector 做many-hot-encoding，C和弦onset的位置為1,5,8 <br>
2.把df的chord(row是每篇吉他譜和弦)欄位與song_chord(吉他譜對應的歌曲做完和弦辨識的欄位)合併成一個欄位<br>
以便切train、test、valid


In [7]:
chord_vector = {
               
        'C:maj':  [0,4,7],
        'Db:maj': [1,5,8],
        'Db:min': [1,4,8],
        'D:maj':  [2,7,9],
        'Eb:maj': [3,7,10],
        'Eb:min': [3,6,10],
        'E:maj':  [4,8,11],
        'E:min':  [4,7,11],
        'F:maj':  [0,5,9],
        'F:min':  [0,4,9],
        'Gb:maj': [1,6,10],
        'Gb:min': [1,6,9],
        'G:maj':  [2,7,11],
        'G:min':  [2,7,10],
        'Ab:maj': [0,3,8],
        'Ab:min': [3,8,11],
        'A:maj':  [1,4,9],
        'A:min':  [0,4,9],
        'Bb:maj': [2,5,10],
        'Bb:min': [1,5,10],
        'B:maj':  [2,6,11],
        'B:min':  [1,6,11]    
   
    }


用chord2vec_df這張大df切train、test、valid

In [8]:
chord2vec_df=df[['chord','song_chord']].stack().reset_index()
chord2vec_df[0]
chord2vec_df=pd.DataFrame({'chord':chord2vec_df[0]})
chord2vec_df.head()

,chord
0,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,..."
1,"[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,..."
2,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m..."
3,"[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:..."
4,"[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,..."


In [12]:
chord2vec_df_train, chord2vec_df_test = train_test_split(chord2vec_df, test_size=0.3, random_state=2019)
chord2vec_df_test,chord2vec_df_valid=train_test_split(chord2vec_df_test, test_size=0.5, random_state=2019)

print('train:',len(chord2vec_df_train))
print('test:',len(chord2vec_df_test))
print('valid:',len(chord2vec_df_valid))

train: 28142
test: 6031
valid: 6031


In [13]:
#chord_word_data:list,存吉他譜與歌曲的和弦進行，一個和弦是一個字，一首歌曲譜是一篇文章
train_data=[]
test_data=[]
valid_data=[]
for row in chord2vec_df_train.chord:
    row_list=[]
    for chord in row:
        if chord in chord_vector:
            row_list.append(chord_vector.get(chord))
    train_data.append(row_list)
for row in chord2vec_df_test.chord:
    row_list=[]
    for chord in row:
        if chord in chord_vector:
            row_list.append(chord_vector.get(chord))
    test_data.append(row_list)
for row in chord2vec_df_valid.chord:
    row_list=[]
    for chord in row:
        if chord in chord_vector:
            row_list.append(chord_vector.get(chord))
    valid_data.append(row_list)

# Chord2Vec需要每個字(和弦)的上下文序列作為訓練資料 
使用dp.make_chord_context(train_data,test_data,valid_data,1)幫和弦配對上下文 <br>
使用prepare_training_data(train_chords,test_chords,valid_chords)把train_set，test_set,valid_set做好

In [14]:
#需要1~2分鐘
start = time.time()
train_chords, test_chords , valid_chords = dp.make_chord_context(train_data,test_data,valid_data,1)
time.time()-start

38.38114953041077

In [15]:
#沒用到的變數先刪掉  節省記憶體
del chord2vec_df_train,chord2vec_df_test,chord2vec_df_valid
gc.collect()

254

In [16]:
del chord2vec_df,df,row,row_list,song_train_merged
gc.collect()

685

In [17]:
del   train_data,test_data,valid_data
gc.collect()

0

# 儲存train_chords、test_chords、valid_chords
train_chords已經做完context配對  <br>
接下來要製作train_set

In [19]:
# out = open("LinearModel_data/0513/train_chords.pkl","wb")
# pickle.dump(train_chords, out)
# out = open("LinearModel_data/0513/test_chords.pkl","wb")
# pickle.dump(test_chords, out)
# out = open("LinearModel_data/0513/valid_chords.pkl","wb")
# pickle.dump(valid_chords, out)
# print('ok')

ok


In [3]:
import pickle

with open('LinearModel_data/train_chords.pkl', 'rb') as pickle_load:
    train_chords = pickle.load(pickle_load)
with open('LinearModel_data/test_chords.pkl', 'rb') as pickle_load:
    test_chords = pickle.load(pickle_load)
with open('LinearModel_data/valid_chords.pkl', 'rb') as pickle_load:
    valid_chords = pickle.load(pickle_load)

In [4]:
def prepare_training_data(train_chords,test_chords,valid_chords):
    print('Loading data ...')
 
    train_set = dp.generate_binary_vectors(train_chords)
    # input_train, target_train = train_set
    del  train_chords
    gc.collect()
    test_set = dp.generate_binary_vectors(test_chords)
    del  test_chords
    gc.collect()
    valid_set = dp.generate_binary_vectors(valid_chords)
    del  valid_chords
    gc.collect()
    # input_valid, target_valid = valid_set
    
    data_size = len(train_set[0])
    data_size_valid = len(valid_set[0])
    data_size_te = len(test_set[0])

    total_batch = int(data_size / batch_size)
    total_batch_valid = int(data_size_valid / batch_size)
    total_batch_test = int(data_size_te / batch_size)

    return train_set, test_set, valid_set, total_batch, total_batch_test, total_batch_valid



In [5]:
# Parameters
learning_rate = 0.001
training_epochs = 200
batch_size = 128
display_step = 1

# Network Parameters
D = 500 # 1st layer number of features
NUM_NOTES = 12 #nums of Vocab

# tf Graph input
input = tf.placeholder("float", [None, NUM_NOTES])
target = tf.placeholder("float", [None, NUM_NOTES])

# Create model
def linear(input, weights):
    hidden = tf.matmul( 
        tf.truediv(
            input, 
            tf.maximum(
                1.0,
                tf.reduce_sum(
                    input, 
                    1, 
                    keep_dims=True
                )
            ) 
        ) ,
        weights['hidden']
    ) #+ bias['hidden1']
    out_layer = tf.matmul(
        hidden,
        weights['out'])# + bias['out']
    return out_layer

# Store layers weight & bias
weights = {
    'hidden': tf.Variable(tf.random_normal([NUM_NOTES, D]),name='embedding'),
    'out': tf.Variable(tf.random_normal([D, NUM_NOTES]))
}

bias = {
    'hidden': tf.Variable(tf.random_normal([D])),
    'out': tf.Variable(tf.random_normal([NUM_NOTES]))
}
embedding_initial=[]

In [9]:

# loads the embeddings variable from the tensorflow checkpoint
def load_embeddings(checkpoint_path):
    with tf.Session() as session:
        if checkpoint_path:
            saver = tf.train.import_meta_graph(checkpoint_path+'/linear_D1024.ckpt-6.meta')
            saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
          
            return session.run('embedding:0')
def get_batch(data_set,id, stoch=False):
    if stoch:
        transpose_data_set = list(map(list, zip(*data_set)))
        batch = random.sample(transpose_data_set, batch_size)
        batch_input,batch_target = list(map(list, zip(*batch)))
        return batch_input,batch_target
    batch_id = id + 1
    input, target = data_set
    return input[(batch_id * batch_size - batch_size):(batch_id * batch_size)], target[(batch_id * batch_size - batch_size):(batch_id * batch_size)]


def train(checkpoint_path='save_models/linear/0507/linear_D1024.ckpt',load_model=None,print_train=True, print_test=True):
    train_set, test_set, valid_set, total_batch, total_batch_test, total_batch_valid = prepare_training_data(train_chords,test_chords,valid_chords)
    data_size = len(train_set[0])

    # Construct model
    print('Create model ...')
    
    pred = linear(input, weights)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=target),1))


    #optimizer = tf.train.AdamOptimizer(epsilon=1e-01,learning_rate=learning_rate).minimize(cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.initialize_all_variables()
    saver = tf.train.Saver(tf.all_variables(),max_to_keep=1)
    
    input_valid, target_valid = valid_set

    # Launch the graph
    print('Start training ...')
    with tf.Session() as sess:
        
        # checkpoint = False #tf.train.get_checkpoint_state('save_models/linear')
        # if checkpoint and tf.gfile.Exists(checkpoint.model_checkpoint_path):
        #     print("Reading model parameters from %s" % checkpoint.model_checkpoint_path)
        #     saver.restore(sess, checkpoint.model_checkpoint_path)
        #     best_val_loss = sess.run(cost, feed_dict={input: input_valid, target: target_valid})
        
        print('初始化tf變數...')
        sess.run(init)
        best_val_loss = np.inf
        embedding_initial=sess.run('embedding:0')
        
        if  print_test:
            # Training cycle
            previous_eval_loss = []
            best_val_epoch = -1
            strikes = 0
            print('Start Training cycle...')
            for epoch in range(training_epochs):
                avg_cost = 0.
                total_batch = int(data_size/batch_size)
                # Loop over all batches
                if epoch %10==0:print('Epoch:',epoch,'total_batch:',total_batch)
                    
                for i in range(total_batch):
                    batch_x, batch_y = get_batch(train_set,i)
                    # Run optimization op (backprop) and cost op (to get loss value)
                    _, c, out = sess.run([optimizer, cost, pred], feed_dict={input: batch_x,
                                                                 target: batch_y})

                    # Compute average loss
                    avg_cost += c / total_batch
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%d' % (epoch+1), "cost=", \
                        "{:.9f}".format(avg_cost))
                c_valid = sess.run(cost, feed_dict={input: input_valid, target: target_valid})
                print("Valid error %4f" % (c_valid))
                previous_eval_loss.append(c_valid)
                improve_valid = previous_eval_loss[-1] < best_val_loss

                if improve_valid:
                    best_val_loss = previous_eval_loss[-1]
                    best_val_epoch = epoch
                    # Save checkpoint.
                    saver.save(sess, checkpoint_path,global_step=epoch)
                else:
                    strikes += 1
#                 if strikes > 3:
#                     break
            print("Optimization Finished!")

        input_test, target_test = test_set
        c_test = sess.run(cost, feed_dict={input: input_test, target: target_test})


        print("Test error %.9f" % (c_test))
        print("Best validation %.9f" % (best_val_loss))
        
        print('訓練完成')
        

In [10]:
train()

Loading data ...
Create model ...
Start training ...
初始化tf變數...
Start Training cycle...
Epoch: 0 total_batch: 39669
Epoch: 1 cost= 6.666183077
Valid error 6.045089
Epoch: 2 cost= 6.024536747
Valid error 6.045136
Epoch: 3 cost= 6.024531387
Valid error 6.045253
Epoch: 4 cost= 6.024531363
Valid error 6.045240
Epoch: 5 cost= 6.024531283
Valid error 6.045240
Epoch: 6 cost= 6.024531191
Valid error 6.045244
Epoch: 7 cost= 6.024531089
Valid error 6.045189
Epoch: 8 cost= 6.024531005
Valid error 6.045182
Epoch: 9 cost= 6.024530902
Valid error 6.045063
Epoch: 10 cost= 6.024530801
Valid error 6.045058
Epoch: 10 total_batch: 39669
Epoch: 11 cost= 6.024530709
Valid error 6.045064
Epoch: 12 cost= 6.024530613
Valid error 6.045083
Epoch: 13 cost= 6.024530524
Valid error 6.045098
Epoch: 14 cost= 6.024530441
Valid error 6.045105
Epoch: 15 cost= 6.024530351
Valid error 6.045105
Epoch: 16 cost= 6.024530233
Valid error 6.045098
Epoch: 17 cost= 6.024530146
Valid error 6.045101
Epoch: 18 cost= 6.024530066
Val

Epoch: 156 cost= 6.024517188
Valid error 6.043430
Epoch: 157 cost= 6.024517088
Valid error 6.043426
Epoch: 158 cost= 6.024517003
Valid error 6.043289
Epoch: 159 cost= 6.024516896
Valid error 6.043286
Epoch: 160 cost= 6.024516808
Valid error 6.043326
Epoch: 160 total_batch: 39669
Epoch: 161 cost= 6.024516715
Valid error 6.043332
Epoch: 162 cost= 6.024516622
Valid error 6.043296
Epoch: 163 cost= 6.024516536
Valid error 6.043258
Epoch: 164 cost= 6.024516452
Valid error 6.043282
Epoch: 165 cost= 6.024516354
Valid error 6.043258
Epoch: 166 cost= 6.024516242
Valid error 6.043640
Epoch: 167 cost= 6.024516160
Valid error 6.043642
Epoch: 168 cost= 6.024516076
Valid error 6.043605
Epoch: 169 cost= 6.024515997
Valid error 6.043612
Epoch: 170 cost= 6.024515889
Valid error 6.043629
Epoch: 170 total_batch: 39669
Epoch: 171 cost= 6.024515780
Valid error 6.043631
Epoch: 172 cost= 6.024515687
Valid error 6.043632
Epoch: 173 cost= 6.024515604
Valid error 6.043543
Epoch: 174 cost= 6.024515521
Valid error

# 訓練完後神經網路隱藏層的權重可以拿來當每個和弦的Embedding
embeddings為lookup table,維度12X1024，12是訓練神經網路時先把每個和弦當作一個字並用12-vecotor來表徵，1024是隱藏層節點數 <br>
例:C和弦onset的位置為[0, 4, 7] <br>
則拿出lookup table位置為[0,4,7]的三個row
然後把3個row加總起來變成一個1X1024維度的陣列當做C和弦的Embedding


In [2]:
# loads the embeddings variable from the tensorflow checkpoint
def load_embeddings(checkpoint_path):
    with tf.Session() as session:
        if checkpoint_path:
            saver = tf.train.import_meta_graph(checkpoint_path+'/linear_D1024.ckpt-164.meta')
            saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
          
            return session.run('embedding:0')

In [5]:
embeddings_trained=load_embeddings('save_models/linear')
pd.DataFrame(embeddings_trained).head()

INFO:tensorflow:Restoring parameters from save_models/linear\linear_D1024.ckpt-6


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.885763,-0.698374,-0.985356,-0.168804,-1.462866,-1.173522,0.771879,0.330421,0.843977,0.709352,...,0.765034,0.081080,-0.951852,-1.100772,0.257062,1.653568,1.865348,-0.297716,-0.945742,2.476066
1,-0.564719,-0.497007,-1.393687,-0.728143,-0.716429,-2.432258,-1.967033,0.184841,1.747492,-0.963003,...,-0.602678,0.996899,-0.049030,-2.182903,-0.445242,0.466168,1.033728,0.939471,0.534260,1.432744
2,-1.147131,0.338825,0.190830,-0.465612,0.293960,0.851809,0.926844,0.542515,-0.390931,1.758892,...,-0.351451,-0.998305,-0.953686,1.127539,0.822034,-0.498805,-1.194230,-0.379888,-0.332850,-1.293034
3,-0.448611,0.408163,-0.106104,-1.350085,-1.208089,-0.686389,0.676287,0.910190,1.533031,-0.017758,...,-0.125893,1.198886,-0.058276,-2.070835,0.557820,-1.547150,-0.506189,-0.566411,0.090274,-1.076528
4,-1.008666,0.746809,-1.696258,1.211193,-1.171438,0.216586,-0.923916,0.035863,0.850063,-0.356309,...,-0.676470,-1.356658,3.318455,-0.204378,2.091613,0.429440,0.789041,-1.254088,-0.519818,1.526186


In [29]:
index=chord_vector.get('C:maj')
Cmaj_Embedding=embeddings_trained[index].sum(axis=0).reshape(1, 1024)
pd.DataFrame(Cmaj_Embedding)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-2.475685,0.913832,-3.746457,1.352629,-1.555308,-1.950082,-0.153541,0.78741,0.735403,-0.927289,...,-0.100187,-1.417388,1.308784,-2.470891,1.78009,2.126603,1.601808,-2.80539,-2.189459,3.402207


# 用pretained好的chord2vec表徵每首曲子的每個和弦
C和弦是chord2vec lookup表的[0,4,7]rows的總和  <br>
把每個和弦的vector總和算出來變成1X500並做出一張表，之後查這張表的和弦對應的row以找出一個和弦的vector總和

In [8]:
chord_vector = {
               
        'C:maj':  [0,4,7],
        'Db:maj': [1,5,8],
        'Db:min': [1,4,8],
        'D:maj':  [2,7,9],
        'Eb:maj': [3,7,10],
        'Eb:min': [3,6,10],
        'E:maj':  [4,8,11],
        'E:min':  [4,7,11],
        'F:maj':  [0,5,9],
        'F:min':  [0,4,9],
        'Gb:maj': [1,6,10],
        'Gb:min': [1,6,9],
        'G:maj':  [2,7,11],
        'G:min':  [2,7,10],
        'Ab:maj': [0,3,8],
        'Ab:min': [3,8,11],
        'A:maj':  [1,4,9],
        'A:min':  [0,4,9],
        'Bb:maj': [2,5,10],
        'Bb:min': [1,5,10],
        'B:maj':  [2,6,11],
        'B:min':  [1,6,11]    
   
    }
chord_index={
    'C:maj':  1,
    'Db:maj': 2,
    'Db:min': 3,
    'D:maj':  4,
    'Eb:maj': 5,
    'Eb:min': 6,
    'E:maj':  7,
    'E:min':  8,
    'F:maj':  9,
    'F:min':  10,
    'Gb:maj': 11,
    'Gb:min': 12,
    'G:maj':  13,
    'G:min':  14,
    'Ab:maj': 15,
    'Ab:min': 16,
    'A:maj':  17,
    'A:min':  18,
    'Bb:maj': 19,
    'Bb:min': 20,
    'B:maj':  21,
    'B:min':  22    

}

In [6]:
embeddings_trained=load_embeddings('./save_models/linear/0507')

INFO:tensorflow:Restoring parameters from ./save_models/linear/0507\linear_D1024.ckpt-164


In [9]:
Chord_embedding_sum_lookup=np.array([])
Chord_embedding_sum_lookup=np.append(Chord_embedding_sum_lookup,np.zeros(500))
for chord in chord_vector:
    index=chord_vector.get(chord)
    chord_Embedding_sum=embeddings_trained[index].sum(axis=0).reshape(1, 500)
    Chord_embedding_sum_lookup=np.append(Chord_embedding_sum_lookup,chord_Embedding_sum)
Chord_embedding_sum_lookup=Chord_embedding_sum_lookup.reshape(23,500)  

In [10]:
import pickle

out = open("../Question_Answering_Structure/QA_data/Chord_embedding_sum_lookup","wb")
pickle.dump(Chord_embedding_sum_lookup, out)
print('ok')

ok
